## 공공데이터 
공공데이터 실험
- 대학교, 카페, 공원, 금리, 주택가격지수, 은행, 쇼핑몰 등등 : 성적 감소
- 구청 : 성적 상승 

구청만 피쳐로 사용했기에 구청의 링크만 냅두도록 하겠습니다. 
- 링크 : http://www.juso.go.kr/addrlink/addressBuildDevNew.do?menu=fclts

**첨부한 자료의 위,경도를 바꾼 코드는 첨부한 public_code.zip을 보시면 됩니다. **

In [24]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [25]:
import os
print(os.listdir())

['apartment_school.csv', '(0119)seouluniv_entrance.csv', '(0119)seoul_large_shop.csv', 'Apartment_subway.csv', 'SEOUL_SOL.csv', 'submission.csv', 'train.csv', 'test.csv', 'Subways.csv', 'min_distance_apartment.csv', 'apartment_public.csv', 'Apartment_ga_hd.csv', '[2019.01.21] hangjeong_info.ipynb', 'latlng_seoul_busan_public.csv', 'BUSAN_SOL.csv', '[2019.01.21] GongGong.ipynb', '.ipynb_checkpoints', 'Schools.csv']


In [26]:
public_info = pd.read_csv("latlng_seoul_busan_public.csv",encoding='UTF-8')
public_info.head()

,유형,상세분류,시군구코드,도로명코드,도로명주소,기관명,위치X,위치Y,전화번호,위도,경도
0,경찰서,경찰서,26440,3350851,부산광역시 강서구 명지국제1로 305,부산강서경찰서,1.128138e+06,1.681014e+06,182,35.116154,128.906165
1,경찰서,경찰서,26410,2000010,부산광역시 금정구 중앙대로 1819,부산금정경찰서,1.144807e+06,1.695755e+06,182,35.246765,129.091625
2,경찰서,경찰서,26710,2006018,부산광역시 기장군 일광면 기장대로 692,부산기장경찰서,1.156926e+06,1.697027e+06,182,35.256405,129.225014
3,경찰서,경찰서,26290,4193469,부산광역시 남구 황령대로319번가길 81,부산남부경찰서,1.144794e+06,1.685353e+06,182,35.153007,129.089649
4,경찰서,경찰서,26260,3130023,부산광역시 동래구 명륜로 70,부산동래경찰서,1.144254e+06,1.690629e+06,182,35.200645,129.084649


In [27]:
public_info = public_info[['상세분류','기관명','위도','경도']]
public_info.columns = ['type','name','latitude','longitude']

In [28]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
df_all = pd.concat([train,test])

apartment = df_all.groupby(['apartment_id'])['latitude','longitude','address_by_law'].mean().reset_index()
apartment.head()

,apartment_id,latitude,longitude,address_by_law
0,3,37.517649,127.012646,1165010600
1,4,37.535133,127.092901,1121510300
2,5,37.536370,127.050762,1120011500
3,24,35.160082,129.022599,2623011100
4,26,37.500199,127.045774,1168010100


In [29]:
from math import cos, asin, sqrt
# 단위는 km로 나옴.
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [30]:
public_info_seoul = public_info[public_info['latitude']//1 == 37].reset_index(drop=True)
public_info_busan = public_info[public_info['latitude']//1 == 35].reset_index(drop=True)
apartment_seoul = apartment[apartment['latitude']//1 == 37].reset_index(drop=True)
apartment_busan = apartment[apartment['latitude']//1 == 35].reset_index(drop=True)

In [31]:
public_info_seoul['type'].unique()

array(['경찰서', '지방청', '지구대', '파출소', '시군구', '시도', '읍면동', '기술센터', '보건지소',
       '고용센터', '위원회', '지청', '보건소', '대학', '세관', '지소', '세무서', '고등학교',
       '지방국세청', '보훈', '등기소', '민원실', '법원', '중학교', '초등학교', '우정청', '취급국',
       '우체국', '집중국', '운전면허시험장'], dtype=object)

In [44]:
public_info_seoul_univ = public_info_seoul[public_info_seoul['type']=='대학']
public_info_busan_univ = public_info_busan[public_info_busan['type']=='대학']

In [45]:
public_info_seoul_univ.head()

,type,name,latitude,longitude
683,대학,중앙대학교,37.504685,126.953857
758,대학,이화여자대학교,37.564555,126.950320
759,대학,인덕대학교,37.630344,127.054500
760,대학,장로회신학대학교,37.550265,127.103134
761,대학,정석대학,37.546217,126.813886


In [46]:
public_info_seoul_univ['name'].unique()

array(['중앙대학교', '이화여자대학교', '인덕대학교', '장로회신학대학교', '정석대학', '총신대학교',
       '추계예술대학교', '케이씨대학교', '한국방송통신대학교', '한국성서대학교', '한국예술종합학교',
       '한국외국어대학교', '한국체육대학교', '한국폴리텍 I 대학 서울강서캠퍼스', '한국폴리텍 I 대학 서울정수캠퍼스',
       '한성대학교', '한양대학교', '한양사이버대학교', '한양여자대학교', '홍익대학교', '가톨릭대학교',
       '감리교신학대학교', '건국대학교', '경기대학교', '경희대학교', '경희사이버대학교', '고려대학교',
       '고려사이버대학교', '광운대학교', '국민대학교', '덕성여자대학교', '동국대학교', '동덕여자대학교',
       '동양미래대학교', '디지털서울문화예술대학교', '명지대학교', '명지전문대학', '배화여자대학교',
       '사이버한국외국어대학교', '삼육대학교', '삼육보건대학교', '상명대학교', '서강대학교', '서경대학교',
       '서울과학기술대학교', '서울교육대학교', '서울기독대학교', '서울대학교', '서울디지털대학교', '서울사이버대학교',
       '서울시립대학교', '서울여자간호대학교', '서울여자대학교', '서울한영대학교', '서일대학교', '성공회대학교',
       '성균관대학교', '성신여자대학교', '세종대학교', '세종사이버대학교', '숙명여자대학교', '숭실대학교',
       '숭실사이버대학교', '숭의여자대학교', '연세대학교', '한국열린사이버대학교'], dtype=object)

In [47]:
univ_list = ['중앙대학교','이화여자대학교','한국외국어대학교','한국체육대학교','한국폴리텍 I 대학 서울강서캠퍼스','한국폴리텍 I 대학 서울정수캠퍼스',
             '한성대학교','한양대학교','한양여자대학교','홍익대학교','가톨릭대학교','건국대학교','경기대학교','경희대학교','고려대학교','광운대학교',
            '국민대학교','덕성여자대학교','동국대학교','동덕여자대학교','명지대학교','배화여자대학교','상명대학교','서강대학교','서경대학교','서울과학기술대학교','서울교육대학교',
            '서울대학교','서울시립대학교','서울여자대학교','성균관대학교','성신여자대학교','세종대학교','숙명여자대학교','숭실대학교','연세대학교']
public_info_seoul_univ['name'] = public_info_seoul_univ['name'].apply(lambda x: x if x in univ_list else np.nan)

In [48]:
public_info_seoul_univ = public_info_seoul_univ.dropna(axis=0)
public_info_seoul_univ.shape

(37, 4)

In [49]:
public_info_seoul_univ

,type,name,latitude,longitude
683,대학,중앙대학교,37.504685,126.953857
758,대학,이화여자대학교,37.564555,126.950320
772,대학,한국외국어대학교,37.598297,127.057942
773,대학,한국체육대학교,37.519886,127.132018
774,대학,한국폴리텍 I 대학 서울강서캠퍼스,37.549242,126.842445
775,대학,한국폴리텍 I 대학 서울정수캠퍼스,37.529975,126.996817
776,대학,한성대학교,37.583236,127.010390
777,대학,한양대학교,37.554504,127.046611
779,대학,한양여자대학교,37.557918,127.049322
801,대학,홍익대학교,37.552519,126.924991


In [50]:
#서울 학교갯수
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
public_info_seoul_univ_lat_lon = public_info_seoul_univ[['latitude','longitude']].reset_index(drop=True)

count_list1 = []


for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0


        for j in range(0,public_info_seoul_univ_lat_lon.shape[0]):
            for lat2,lon2 in [public_info_seoul_univ_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1.2:
                    count1 += 1
                else:
                    pass
        

                    
        count_list1.append(count1)
count_list1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,


In [51]:
apartment_seoul['univ_1,2'] = count_list1

In [52]:
#부산 학교갯수
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
public_info_busan_univ_lat_lon = public_info_busan_univ[['latitude','longitude']].reset_index(drop=True)

count_list1 = []


for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0


        for j in range(0,public_info_busan_univ_lat_lon.shape[0]):
            for lat2,lon2 in [public_info_busan_univ_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1.2:
                    count1 += 1
                else:
                    pass
        

                    
        count_list1.append(count1)
count_list1

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 3,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 3,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 0,


In [53]:
apartment_busan['univ_1,2'] = count_list1

### 관공서

In [54]:
public_info_seoul['type'].unique()

array(['경찰서', '지방청', '지구대', '파출소', '시군구', '시도', '읍면동', '기술센터', '보건지소',
       '고용센터', '위원회', '지청', '보건소', '대학', '세관', '지소', '세무서', '고등학교',
       '지방국세청', '보훈', '등기소', '민원실', '법원', '중학교', '초등학교', '우정청', '취급국',
       '우체국', '집중국', '운전면허시험장'], dtype=object)

In [55]:
public_info_seoul[public_info_seoul['type']=='시군구']

,type,name,latitude,longitude
264,시군구,강남구청,37.517970,127.047051
265,시군구,강동구청,37.530161,127.123792
266,시군구,강북구청,37.639185,127.025450
267,시군구,강서구청,37.550963,126.849549
268,시군구,관악구청,37.478211,126.951501
269,시군구,광진구청,37.538540,127.081912
270,시군구,구로구청,37.495511,126.888289
271,시군구,금천구청,37.457066,126.896037
272,시군구,노원구청,37.654519,127.056297
273,시군구,도봉구청,37.668679,127.047066


In [56]:
public_info_seoul_si = public_info_seoul[public_info_seoul['type']=='시군구']
public_info_busan_si = public_info_busan[public_info_busan['type']=='시군구']

In [57]:
#서울 구청 거리
apartment_lat_lon = apartment_seoul[['latitude','longitude']].reset_index(drop=True)
public_info_seoul_univ_lat_lon = public_info_seoul_si[['latitude','longitude']].reset_index(drop=True)

count_list1 = []


for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0


        for j in range(0,public_info_seoul_univ_lat_lon.shape[0]):
            for lat2,lon2 in [public_info_seoul_univ_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1:
                    count1 += 1
                else:
                    pass
        

                    
        count_list1.append(count1)
count_list1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [58]:
apartment_seoul['public_1'] = count_list1

In [59]:
#부산 학교갯수
apartment_lat_lon = apartment_busan[['latitude','longitude']].reset_index(drop=True)
public_info_busan_univ_lat_lon = public_info_busan_si[['latitude','longitude']].reset_index(drop=True)

count_list1 = []


for i in range(0,apartment_lat_lon.shape[0]):    
    for lat1,lon1 in [apartment_lat_lon.loc[i].values]:
        count1 = 0


        for j in range(0,public_info_busan_univ_lat_lon.shape[0]):
            for lat2,lon2 in [public_info_busan_univ_lat_lon.loc[j].values]:
                if distance(lat1,lon1,lat2,lon2) <= 1.2:
                    count1 += 1
                else:
                    pass
        

                    
        count_list1.append(count1)
count_list1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [60]:
apartment_busan['public_1'] = count_list1
apartment_busan['public_1'].describe()

count    1281.000000
mean        0.248244
std         0.439334
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         2.000000
Name: public_1, dtype: float64

In [61]:
apartment = pd.concat([apartment_seoul,apartment_busan])

In [62]:
apartment.to_csv("apartment_public.csv",index=False)